# my-zero-to-gbm-proj-assign

For this competition I am taking a running competiton in Kaggle which is supposed by end by Sept 28 2021. I am trying to participate in that competition.  Below is my own program written based on the foundations of the stock market volataluty computations help provided by Optiver company to all participants as the domain is not known to all. The base given as a "book example". After that everything else is my self work in determing the Imputing strategy, Feature engineering for Order data and trade data 

## Optiver Realized Volatility Prediction

This dataset contains stock market data relevant to the practical execution of trades in the financial markets. In particular, it includes order book snapshots and executed trades. With one second resolution, it provides a uniquely fine grained look at the micro-structure of modern financial markets.

This is a code competition where only the first few rows of the test set are available for download. The rows that are visible are intended to illustrate the hidden test set format and folder structure. The remainder will only be available to your notebook when it is submitted. The hidden test set contains data that can be used to construct features to predict roughly 150,000 target values. Loading the entire dataset will take slightly more than 3 GB of memory, by our estimation.

This is also a forecasting competition, where the final private leaderboard will be determined using data gathered after the training period closes, which means that the public and private leaderboards will have zero overlap. During the active training stage of the competition a large fraction of the test data will be filler, intended only to ensure the hidden dataset has approximately the same size as the actual test data. The filler data will be removed entirely during the forecasting phase of the competition and replaced with real

In [ ]:
!pip install jovian --upgrade --quiet

In [ ]:
import jovian

#### Data Description

book_[train/test].parquet A parquet file partitioned by stock_id. Provides order book data on the most competitive buy and sell orders entered into the market. The top two levels of the book are shared. The first level of the book will be more competitive in price terms, it will then receive execution priority over the second level.

stock_id - ID code for the stock. Not all stock IDs exist in every time bucket. Parquet coerces this column to the categorical data type when loaded; you may wish to convert it to int8.
time_id - ID code for the time bucket. Time IDs are not necessarily sequential but are consistent across all stocks.
seconds_in_bucket - Number of seconds from the start of the bucket, always starting from 0.
bid_price[1/2] - Normalized prices of the most/second most competitive buy level.
ask_price[1/2] - Normalized prices of the most/second most competitive sell level.
bid_size[1/2] - The number of shares on the most/second most competitive buy level.
ask_size[1/2] - The number of shares on the most/second most competitive sell level.
trade_[train/test].parquet A parquet file partitioned by stock_id. Contains data on trades that actually executed. Usually, in the market, there are more passive buy/sell intention updates (book updates) than actual trades, therefore one may expect this file to be more sparse than the order book.

stock_id - Same as above.
time_id - Same as above.
seconds_in_bucket - Same as above. Note that since trade and book data are taken from the same time window and trade data is more sparse in general, this field is not necessarily starting from 0.
price - The average price of executed transactions happening in one second. Prices have been normalized and the average has been weighted by the number of shares traded in each transaction.
size - The sum number of shares traded.
order_count - The number of unique trade orders taking place.
train.csv The ground truth values for the training set.

stock_id - Same as above, but since this is a csv the column will load as an integer instead of categorical.
time_id - Same as above.
target - The realized volatility computed over the 10 minute window following the feature data under the same stock/time_id. There is no overlap between feature and target data. You can find more info in our tutorial notebook.
test.csv Provides the mapping between the other data files and the submission file. As with other test files, most of the data is only available to your notebook upon submission with just the first few rows available for download.

stock_id - Same as above.
time_id - Same as above.
row_id - Unique identifier for the submission row. There is one row for each existing time ID/stock ID pair. Each time window is not necessarily containing every individual stock.
sample_submission.csv - A sample submission file in the correct format.

row_id - Same as in test.csv.
target - Same definition as in train.csv. The benchmark is using the median target value from train.csv.

In [ ]:
# Execute this to save new versions of the notebook
#jovian.commit(project="my-zero-to-gbm-proj-assign")
jovian.commit(filename="my-zero-to-gbm-proj-assign.ipynb")

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
#train = pd.read_csv('../../../data/optiver-realized-volatility-prediction/train.csv')
train = pd.read_csv('c:\\optiver-realized-volatility-prediction\\train.csv')

In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428932 entries, 0 to 428931
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   stock_id  428932 non-null  int64  
 1   time_id   428932 non-null  int64  
 2   target    428932 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 9.8 MB


## Points to be addressed before ML modeling

1. stock_id - ID code for the stock. <font color='red'> Not all stock IDs exist in every time bucket </font>. Parquet coerces this column to the categorical data type when loaded;  <font color='red'> you may wish to convert it to int8 </font>
2. We have missing “seconds_in_bucket” field?.A: That means there is no related market activities during the last second. For book data you can also assume the top-2 level book shape stays the same as the last available book update within the gap seconds, or, in another word, <font color='red'> you can forward fill the missing data point for all field in book data.</font>
3. I'm trying to make trade data fixed sized. Since missing seconds_in_bucket implies no trade happening within that one-second window, is it technically correct to resample trade data to 600 seconds and fill it with zeros?. Hi, it is correct to assume 0 for order count and size. Some assumptions are required for the price, though. A price of 0 might cause issues.
4. the trade data at T seconds contains a 1-second aggregation of executed orders between [T, T+1 second]
5. One time_id represents a unique 20-minutes trading window which is consistent across all stocks As an example, let’s say time_id = 1 is representing a window between 1900-01-01 12:00:00 and 1900-01-01 12:20:00, then the book data of all stocks for that time_id are is taken from the same window. The data in the first 10 minutes window is shared with all of you, while the order book data of the second 10-minutes is used to build the target for you to predict. The dataset is rolling in such a way that feature and target data will always have zero overlap. Note that time_id is randomly shuffled, so it will not contain any information other than serving as a bridge between different dataset.


6. We can demonstrate the data structure in below way:
<img src = https://www.optiver.com/wp-content/uploads/2021/05/DataBucketing.png>


7. In our competition, we shared the last snapshot of order book for each second. Imagine you have a time_id starting from 1900-01-01 12:00:00, the book update data on seconds_in_bucket = 1 represents the last snapshot of order book update between 12:00:00 and 12:00:01. Similarly to order book data in terms of granularity, but the trade data represents the aggregation of all individual orders happened within one second.
8. So per stock, under the same time_id, the trade data on seconds_in_bucket = 1 represents the aggregation of all individual executed orders between 12:00:00 and 12:00:01. The size is the sum of the size in each individual order, while the price is aggregated as a volume weighted average price of all trades. A straightforward WAP formula can be found on Investopedia.
9. Q: Why we have missing “seconds_in_bucket” field?
A: That means there is no related market activities during the last second.

For book data you can also assume the top-2 level book shape stays the same as the last available book update within the gap seconds, or, in another word, you can forward fill the missing data point for all field in book data. For trade data, it implies no trade happening within that one-second window. One thing to note that trade data tends to be more sparse than book data in many cases.

10. 


Taking the first row of data, it implies that the realized vol of the target bucket for time_id 5, stock_id 0 is 0.004136. How does the book and trade data in feature bucket look like for us to build signals?

In [3]:
book_example = pd.read_parquet('c:\\optiver-realized-volatility-prediction\\book_train.parquet/stock_id=0')
trade_example =  pd.read_parquet('c:\\optiver-realized-volatility-prediction\\trade_train.parquet/stock_id=0')
stock_id = '0'
book_example = book_example[book_example['time_id']==5]
book_example.loc[:,'stock_id'] = stock_id
trade_example = trade_example[trade_example['time_id']==5]
trade_example.loc[:,'stock_id'] = stock_id

#### Book data snapshot

In [4]:
book_example.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302 entries, 0 to 301
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   time_id            302 non-null    int16  
 1   seconds_in_bucket  302 non-null    int16  
 2   bid_price1         302 non-null    float32
 3   ask_price1         302 non-null    float32
 4   bid_price2         302 non-null    float32
 5   ask_price2         302 non-null    float32
 6   bid_size1          302 non-null    int32  
 7   ask_size1          302 non-null    int32  
 8   bid_size2          302 non-null    int32  
 9   ask_size2          302 non-null    int32  
 10  stock_id           302 non-null    object 
dtypes: float32(4), int16(2), int32(4), object(1)
memory usage: 15.3+ KB


### Trade Book Snapshot

In [5]:
trade_example.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   time_id            40 non-null     int16  
 1   seconds_in_bucket  40 non-null     int16  
 2   price              40 non-null     float32
 3   size               40 non-null     int32  
 4   order_count        40 non-null     int16  
 5   stock_id           40 non-null     object 
dtypes: float32(1), int16(3), int32(1), object(1)
memory usage: 1.2+ KB


##### Realized volatility calculation

our target is to predict short-term realized volatility. Although the order book and trade data for the target cannot be shared, we can still present the realized volatility calculation using the feature data we provided.

As realized volatility is a statistical measure of price changes on a given stock, to calculate the price change we first need to have a stock valuation at the fixed interval (1 second). We will use weighted averaged price, or WAP, of the order book data we provided.

In [6]:
book_example['bid_size'] = (book_example['bid_size1'] + book_example['bid_size2'])
book_example['bid_price'] = (book_example['bid_price1'] * book_example['bid_size1'] + book_example['bid_price2'] * book_example['bid_size2'])/book_example['bid_size']

book_example['ask_size'] = (book_example['ask_size1'] + book_example['ask_size2'])
book_example['ask_price'] = (book_example['ask_price1'] * book_example['ask_size1'] + book_example['ask_price2'] * book_example['ask_size2'])/(book_example['ask_size1'] + book_example['ask_size2'])

book_example['wap'] = (book_example['bid_price'] * book_example['ask_size'] + book_example['ask_price'] * book_example['bid_size']) / (book_example['bid_size'] +  book_example['ask_size'])

book_example['wap1'] = (book_example['bid_price1'] * book_example['ask_size1'] + book_example['ask_price1'] * book_example['bid_size1']) / (book_example['bid_size1'] +  book_example['ask_size1'])

book_example['wap2'] = (book_example['bid_price2'] * book_example['ask_size2'] + book_example['ask_price2'] * book_example['bid_size2']) / (book_example['bid_size2'] +  book_example['ask_size2'])


The WAP of the stock is plotted below

In [7]:
fig = px.line(book_example, x="seconds_in_bucket", y="wap", title='WAP of stock_id_0, time_id_5')
fig.show()

### Log returns

How can we compare the price of a stock between yesterday and today?

The easiest method would be to just take the difference. This is definitely the most intuitive way, however price differences are not always comparable across stocks. For example, let's assume that we have invested \$ 1000 in both stock A and stock B and that stock A moves from  \$ 100 to  \$ 102 and stock B moves from  \$ 10 to  \$ 11. We had a total of 10 shares of A ( \$1000 / \$100=10 ) which led to a profit of  10⋅(\$102−\$100)=\$20  and a total of 100 shares of B that yielded \$100. So the price increase was larger for stock A, although the move was proportionally much larger for stock B.

We can solve the above problem by dividing the move by the starting price of the stock, effectively computing the percentage change in price, also known as the stock return. In our example, the return for stock A was  \$102−\$100/\$100=2% , while for stock B it was  \$11−\$10/\$10=10% . The stock return coincides with the percentage change in our invested capital.

Returns are widely used in finance, however log returns are preferred whenever some mathematical modelling is required. Calling  St  the price of the stock  S  at time  t , we can define the log return between  t1  and  t2  as:

 - r(t1,t2)=log(St2/St1)
 
Usually, we look at log returns over fixed time intervals, so with 10-minute log return we mean  rt=rt−10min,t .

Log returns present several advantages, for example:

- they are additive across time  r(t1,t2) + r(t2,t3) = (rt1,t3) 
- regular returns cannot go below -100%, while log returns are not bounded

Next we will compute the log return

To compute the Log Return, we will simply take the logarithm of the ratio between two consecutive WAP. The first row will have an empty return as the previous book update is unknown, therefore the empty return data point will be dropped.

In [8]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

In [9]:
book_example.loc[:,'log_return'] = log_return(book_example['wap'])


In [10]:
book_example['log_return'] = book_example['log_return'].fillna(book_example['log_return'].mean())

In [11]:
book_example.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,bid_size,bid_price,ask_size,ask_price,wap,wap1,wap2,log_return
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0,5,1.001401,326,1.002317,1.001415,1.001434,1.001390,7.719553e-06
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0,5,1.001401,200,1.002327,1.001424,1.001448,1.001390,8.733591e-06
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0,5,1.001401,200,1.002353,1.001425,1.001448,1.001391,6.300396e-07
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0,5,1.001401,226,1.002347,1.001422,1.001443,1.001391,-2.737289e-06
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0,5,1.001401,226,1.002347,1.001422,1.001443,1.001391,0.000000e+00


In [12]:
#~ means NOT 
# also the null condition is applied because when we .diff() method the first row will have empty value as we cannot have diff of return with respect to the previous time id as the first row is the very first time id
book_example = book_example[~book_example['log_return'].isnull()]

In [13]:
book_example['log_return'].isnull().sum()

0

Let's plot the tick-to-tick return of this instrument over this time bucket

In [14]:
fig = px.line(book_example, x="seconds_in_bucket", y="log_return", title='Log return of stock_id_0, time_id_5')
fig.show()

###Realized volatility

When we trade options, a valuable input to our models is the standard deviation of the stock log returns. The standard deviation will be different for log returns computed over longer or shorter intervals, for this reason it is usually normalized to a 1-year period and the annualized standard deviation is called volatility.

In this competition, you will be given 10 minutes of book data and we ask you to predict what the volatility will be in the following 10 minutes. Volatility will be measured as follows:

We will compute the log returns over all consecutive book updates and we define the realized volatility,  σ , as the squared root of the sum of squared log returns.

\sigma = \sqrt{\sum_{t}r_{t-1, t}^2}
 
Where we use WAP as price of the stock to compute log returns.

We want to keep definitions as simple and clear as possible, so that Kagglers without financial knowledge will not be penalized. So we are not annualizing the volatility and we are assuming that log returns have 0 mean.


The realized vol of stock 0 in this feature bucket, will be:

In [15]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

realized_vol = realized_volatility(book_example['log_return'])
print(f'Realized volatility for stock_id 0 on time_id 5 is {realized_vol}')

Realized volatility for stock_id 0 on time_id 5 is 0.0037818447045652633


### Naive prediction: using past realized volatility as target

A commonly known fact about volatility is that it tends to be autocorrelated. We can use this property to implement a naive model that just "predicts" realized volatility by using whatever the realized volatility was in the initial 10 minutes.

Let's calculate the past realized volatility across the training set to see how predictive a single naive signal can be.

## Feature Engineering with the Data from the Training Data Set provided

We will first get the folder names of all the stock Ids whetre the order booking data and the trading data is stored locally

In [17]:
import glob
list_order_book_file_train = glob.glob('c:\\optiver-realized-volatility-prediction\\book_train.parquet/*')
list_trade_book_file_train = glob.glob('c:\\optiver-realized-volatility-prediction\\trade_train.parquet/*')
#list_order_book_file_train

As the data is partitioned by stock_id (each stock id is one folder), we try to calculcate realized volatility stock by stock and combine them into one target file. Note that the stock id as the partition column is not present if we load the single file so we will remedy that manually. We will reuse the log return and realized volatility functions defined in the previous session.

In [21]:
def compute_wap(df_stock_book):
       df_stock_book['bid_price'] = (df_stock_book['bid_price1'] * df_stock_book['bid_size1'] + df_stock_book['bid_price2'] * df_stock_book['bid_size2'])/(df_stock_book['bid_size1'] + df_stock_book['bid_size2'])
       df_stock_book['bid_size'] = (df_stock_book['bid_size1'] + df_stock_book['bid_size2'])

       df_stock_book['ask_price'] = (df_stock_book['ask_price1'] * df_stock_book['ask_size1'] + df_stock_book['ask_price2'] * df_stock_book['ask_size2'])/(df_stock_book['ask_size1'] + df_stock_book['ask_size2'])
       df_stock_book['ask_size'] = (df_stock_book['ask_size1'] + df_stock_book['ask_size2'])

       df_stock_book['wap'] = (df_stock_book['bid_price'] * df_stock_book['ask_size'] + df_stock_book['ask_price'] * df_stock_book['bid_size']) / (df_stock_book['bid_size'] +  df_stock_book['ask_size'])
       return df_stock_book['wap']

## Data Sampling

The actual training set contains 167 million records. SInce we will not have high end computing resources, i have picked up 25% of the training data

Below I am randomly picking 25% of the of the stock data randomly. For the same random 25 stocks that is pciked i am selecting below the corresponding trade data so that they can be merged  

In [18]:
import random
# # Get current state and store
# state = random.getstate()
# # set current state
# random.setstate(state)
random.seed(42)
list_order_book_file_train_25 = random.sample(list_order_book_file_train, 25)
list_trade_book_file_train_25 = [ l.replace('book','trade') for l in list_order_book_file_train_25]

In [6]:
!pip install pyarrow

In [19]:
pd.io.parquet.get_engine('auto')
pd.io.parquet.PyArrowImpl()

Now for selected 25 Stock Ids we will not parse the parquet file and read the order data and trade data into 2 separate dataframes 

In [22]:
df_order_book_25 = pd.DataFrame()
for file in list_order_book_file_train_25:
     df_stock_book = pd.read_parquet(file)
     df_stock_book['stock_id'] = file.split('=')[1]
     df_order_book_25 = pd.concat([df_order_book_25,df_stock_book])


df_trade_book_25 = pd.DataFrame()
for file in list_trade_book_file_train_25:
     df_stock_book = pd.read_parquet(file)
     df_stock_book['stock_id'] = file.split('=')[1]
     df_trade_book_25 = pd.concat([df_trade_book_25,df_stock_book])


In [23]:
df_order_book_25.stock_id.unique(),df_trade_book_25.stock_id.unique()


(array(['68', '111', '100', '81', '2', '16', '13', '114', '96', '110',
        '73', '9', '55', '109', '61', '39', '101', '97', '85', '126', '14',
        '5', '63', '87', '58'], dtype=object),
 array(['68', '111', '100', '81', '2', '16', '13', '114', '96', '110',
        '73', '9', '55', '109', '61', '39', '101', '97', '85', '126', '14',
        '5', '63', '87', '58'], dtype=object))

In [24]:
df_order_book_25.shape,df_trade_book_25.shape

((36889097, 11), (8110555, 6))

## Selected Sample Size

Our selected sample size
Order data is 36 million
Trade data is 8 million

In [25]:
#new_index
df_order_book_25.reset_index(inplace=True,drop=True)
df_trade_book_25.reset_index(inplace=True,drop=True)


## Imputing

There are several gaps in the Order data and the Trade data

Gap #1 - for each time window there should be ideally 600 records for each stock id. but in reality the data does not have that continuous data. So we have introduce the missing rows for the missing 'seconds in bukcet'

Gap #2 - for all time windows not all stock Ids are traded. But this is tru in reality as well. So we will not fill this

Gap #3 - For all order data there will not be traded data because the BUYER and the SELLER have not come to a single pris where the trade settlement can happen. we can see that Order data is 36 million where as the trade data is 8 million. for this purpose what we will assume the for filling the non traded data

    Size and Count (quanity) of trade will be assumed as zero for the non traded order data
    Price should not be made zero as this will be misleading. so we will assume the last traded prices as the price until the next trade

For Gap #1 - The data Filling Strategy is as follows

    1. first ensure that the Zero "seconds in bucket" record is present and it has no null values. if it has null values then backfill the same from the next available "seconds in bucket" for that time_id and stock_id
    2. Once the "zeroth second" is filled now we will forward fill the remaining rows for each group of STOCK_ID AND TIME_ID
    

We will first merge and the Order data and Trade data into one data frame and start the IMPUTING process

In [28]:
df_order_trade_merged = pd.merge(
    df_order_book_25,
    df_trade_book_25,
    how="outer",
    on=['time_id','stock_id','seconds_in_bucket'],
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate="m:m"
)

In [29]:
df_order_trade_merged.shape

(36889098, 14)

There are 36 million plus records in the marged data. We have used Outer join so that we get all of the Order data and Trade data

In [30]:
#25 Stocks
df_order_trade_merged.stock_id.unique()


array(['100', '101', '109', '110', '111', '114', '126', '13', '14', '16',
       '2', '39', '5', '55', '58', '61', '63', '68', '73', '81', '85',
       '87', '9', '96', '97'], dtype=object)

We will convert Stock_id from object data type to Int8 data type for optimal memory consumption

In [31]:
df_order_trade_merged['stock_id'] = df_order_trade_merged['stock_id'].astype('int8')

we wil create a new index sequence that will reset the "seconds_in_bucket" 

In [26]:
new_index = pd.Index(np.arange(0,600), name="seconds_in_bucket")

For the purpose of imputation and to introduce missing rows, we will first group the data by Time_id and stock_id

In [34]:
grouped = df_order_trade_merged.groupby(['time_id','stock_id'])
#df_order_book_50_idxed.index.levels[2]

In [21]:
grouped.count()

seconds_in_bucket  bid_price1  ask_price1  bid_price2  \
time_id stock_id                                                          
5       2                       583         583         583         583   
        5                       250         250         250         250   
        9                       265         265         265         265   
        13                      502         502         502         502   
        14                      591         591         591         591   
...                             ...         ...         ...         ...   
32767   109                     257         257         257         257   
        110                     157         157         157         157   
        111                     572         572         572         572   
        114                     277         277         277         277   
        126                     217         217         217         217   

                  ask_price2  bid_size1  ask_size1  bid_size2  ask_size2  \
time_id stock_id                                                           
5       2                583        583        583        583        583   
        5                250        250        250        250        250   
        9                265        265        265        265        265   
        13               502        502        502        502        502   
        14               591        591        591        591        591   
...                      ...        ...        ...        ...        ...   
32767   109              257        257        257        257        257   
        110              157        157        157        157        157   
        111              572        572        572        572        572   
        114              277        277        277        277        277   
        126              217        217        217        217        217   

                  price  size  order_count  
time_id stock_id                            
5       2           159   159          159  
        5            36    36           36  
        9            40    40           40  
        13           74    74           74  
        14          276   276          276  
...                 ...   ...          ...  
32767   109          39    39           39  
        110          20    20           20  
        111          80    80           80  
        114          52    52           52  
        126          36    36           36  

[95748 rows x 12 columns]

We will now create a new dataframe with the grouped object above which has Time_Id and Stocck_id as the Index Key....we will intoduce "seconds_in_bucket" as another index. While reindexing with the index sequence all missing rows of "seconds_in_bukcet"  will be created with NaN values

In [35]:
s2=grouped.apply(lambda x : x.set_index('seconds_in_bucket').reindex(new_index))

After creating the Index we will drop the following as normal columns which will be left behind adter reindexing above

In [36]:
s2.drop(columns=['time_id','stock_id'],inplace=True)

In [37]:
s2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 57448800 entries, (5, 2, 0) to (32767, 126, 599)
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   bid_price1   float32
 1   ask_price1   float32
 2   bid_price2   float32
 3   ask_price2   float32
 4   bid_size1    float64
 5   ask_size1    float64
 6   bid_size2    float64
 7   ask_size2    float64
 8   price        float32
 9   size         float64
 10  order_count  float64
dtypes: float32(5), float64(6)
memory usage: 3.9 GB


After imputing above with missing rows for "seconds_in_bucket" we now have a total of 56 million records. 

We now have a multi Index of time_id, stock_id and seconds_in_bucket

In [38]:
print(s2.shape)

(57448800, 11)


In [39]:
print(s2.index)

MultiIndex([(    5,   2,   0),
            (    5,   2,   1),
            (    5,   2,   2),
            (    5,   2,   3),
            (    5,   2,   4),
            (    5,   2,   5),
            (    5,   2,   6),
            (    5,   2,   7),
            (    5,   2,   8),
            (    5,   2,   9),
            ...
            (32767, 126, 590),
            (32767, 126, 591),
            (32767, 126, 592),
            (32767, 126, 593),
            (32767, 126, 594),
            (32767, 126, 595),
            (32767, 126, 596),
            (32767, 126, 597),
            (32767, 126, 598),
            (32767, 126, 599)],
           names=['time_id', 'stock_id', 'seconds_in_bucket'], length=57448800)


In [40]:
s2.head()

bid_price1  ask_price1  bid_price2  \
time_id stock_id seconds_in_bucket                                       
5       2        0                    1.000607    1.000769    1.000526   
                 1                    1.000526    1.000769    1.000445   
                 2                    1.000526    1.000769    1.000364   
                 3                    1.000607    1.000769    1.000526   
                 4                    1.000526    1.000769    1.000445   

                                    ask_price2  bid_size1  ask_size1  \
time_id stock_id seconds_in_bucket                                     
5       2        0                     1.00085      100.0      200.0   
                 1                     1.00085        1.0      200.0   
                 2                     1.00085        1.0      300.0   
                 3                     1.00085      100.0      300.0   
                 4                     1.00085        1.0      300.0   

                                    bid_size2  ask_size2     price   size  \
time_id stock_id seconds_in_bucket                                          
5       2        0                        1.0        1.0  1.000688  101.0   
                 1                      100.0      101.0       NaN    NaN   
                 2                      112.0      101.0       NaN    NaN   
                 3                        1.0        1.0       NaN    NaN   
                 4                      300.0      101.0       NaN    NaN   

                                    order_count  
time_id stock_id seconds_in_bucket               
5       2        0                          2.0  
                 1                          NaN  
                 2                          NaN  
                 3                          NaN  
                 4                          NaN

Below is the amount of null values we have after introducing the missing "seconds" records

In [42]:
s2.isnull().sum()

bid_price1     20559703
ask_price1     20559703
bid_price2     20559703
ask_price2     20559703
bid_size1      20559703
ask_size1      20559703
bid_size2      20559703
ask_size2      20559703
price          49338245
size           49338245
order_count    49338245
dtype: int64

As explained already the imputing strategy we will be filling zero values for the trade data atrributes such as 'size' and 'order count'

In [43]:
s2[['size','order_count']] = s2[['size','order_count']].fillna(0) 


In [44]:
s2.isna().sum()

bid_price1     20559703
ask_price1     20559703
bid_price2     20559703
ask_price2     20559703
bid_size1      20559703
ask_size1      20559703
bid_size2      20559703
ask_size2      20559703
price          49338245
size                  0
order_count           0
dtype: int64

Below we will know for each time_id and stock_id which zero seconds record is having null values in their attributes

Only price attribute in trading data is having null values in zero seconds record. Size and Count are already filled with zeros.

In the order booking data all the 8 attributes have values with no Nulls

In [45]:
s2.loc(axis=0)[:,:,0].isna().sum()


bid_price1         0
ask_price1         0
bid_price2         0
ask_price2         0
bid_size1          0
ask_size1          0
bid_size2          0
ask_size2          0
price          75233
size               0
order_count        0
dtype: int64

In [46]:
s2.loc(axis=0)[:,:,0][s2.loc(axis=0)[:,:,0]['price'].isna()]

bid_price1  ask_price1  bid_price2  \
time_id stock_id seconds_in_bucket                                       
5       5        0                    0.999749    1.001348    0.999655   
        9        0                    1.003797    1.005181    1.003755   
        14       0                    1.000643    1.000710    1.000575   
        16       0                    1.001464    1.002419    1.001401   
        39       0                    1.000377    1.000503    1.000314   
...                                        ...         ...         ...   
32767   96       0                    0.999584    0.999949    0.999563   
        101      0                    1.000047    1.000142    0.999953   
        109      0                    0.999975    1.000370    0.999926   
        114      0                    0.999917    1.000050    0.999850   
        126      0                    0.999949    1.000514    0.999846   

                                    ask_price2  bid_size1  ask_size1  \
time_id stock_id seconds_in_bucket                                     
5       5        0                    1.001361      308.0        5.0   
        9        0                    1.005223      333.0      124.0   
        14       0                    1.000778       53.0      100.0   
        16       0                    1.002547        7.0        1.0   
        39       0                    1.000566       37.0      114.0   
...                                        ...        ...        ...   
32767   96       0                    0.999970      100.0      100.0   
        101      0                    1.000237       56.0      100.0   
        109      0                    1.000420      100.0      300.0   
        114      0                    1.000083      100.0      160.0   
        126      0                    1.000617      202.0      276.0   

                                    bid_size2  ask_size2  price  size  \
time_id stock_id seconds_in_bucket                                      
5       5        0                       11.0       11.0    NaN   0.0   
        9        0                      100.0       41.0    NaN   0.0   
        14       0                      257.0      201.0    NaN   0.0   
        16       0                        9.0       20.0    NaN   0.0   
        39       0                      312.0        2.0    NaN   0.0   
...                                       ...        ...    ...   ...   
32767   96       0                      100.0      100.0    NaN   0.0   
        101      0                      189.0      125.0    NaN   0.0   
        109      0                       71.0      100.0    NaN   0.0   
        114      0                        1.0      100.0    NaN   0.0   
        126      0                       20.0        2.0    NaN   0.0   

                                    order_count  
time_id stock_id seconds_in_bucket               
5       5        0                          0.0  
        9        0                          0.0  
        14       0                          0.0  
        16       0                          0.0  
        39       0                          0.0  
...                                         ...  
32767   96       0                          0.0  
        101      0                          0.0  
        109      0                          0.0  
        114      0                          0.0  
        126      0                          0.0  

[75233 rows x 11 columns]

for those zero seconds records we, as explained already, we need to do backfill first. for that we need to first find the first available values for each of the time_id and stock_id gorup. Then fill that in the main dataframe

In [47]:
bf = s2.groupby(['time_id','stock_id'])

In [48]:
bf['price'].first()

time_id  stock_id
5        2           1.000688
         5           1.001091
         9           1.005213
         13          1.000129
         14          1.000811
                       ...   
32767    109         0.999982
         110         0.999921
         111         1.000101
         114         0.999952
         126         1.000481
Name: price, Length: 95748, dtype: float32

In [50]:
s2.loc[s2.index.get_level_values(2)==0,'price'] = bf['price'].first()

In [51]:
s2.isna().sum()

bid_price1     20559703
ask_price1     20559703
bid_price2     20559703
ask_price2     20559703
bid_size1      20559703
ask_size1      20559703
bid_size2      20559703
ask_size2      20559703
price          49263012
size                  0
order_count           0
dtype: int64

Earlier we had 49338245 records where we had price enpty. Now after filling the zero seconds records along with backfilling from the next available value, the number of records which has null values for price has reduced to 49263012

In [52]:
s2.loc(axis=0)[:,:,0].isna().sum()

bid_price1     0
ask_price1     0
bid_price2     0
ask_price2     0
bid_size1      0
ask_size1      0
bid_size2      0
ask_size2      0
price          0
size           0
order_count    0
dtype: int64

Now we have zero seconds record will no null values. By ensuring this, no values will get propogated from one group of time_id and stock_id to another as the first record in each group (which is zero second record) is now having no null values

Next we will forward fill the values for empty rows with previously available rows for each time_id and stock_id

In [54]:
#s2.loc[:,:,1:] = s2.loc[:,:,1:].fillna(method='ffill')
#s2.loc[:,:,1:].fillna(method='ffill',inplace=True)
s2.fillna(method='ffill',inplace=True)

In [56]:
s2.isna().sum()

bid_price1     0
ask_price1     0
bid_price2     0
ask_price2     0
bid_size1      0
ask_size1      0
bid_size2      0
ask_size2      0
price          0
size           0
order_count    0
dtype: int64

Again for the purpose of svvaing the memory as we are delaing with huse datasets, we will reset the data type for each attribute to the minimum possible

In [57]:
s2['order_count'] = s2['order_count'].astype('int16')
s2[['bid_size1','ask_size1','bid_size2','ask_size2','size']] = s2[['bid_size1','ask_size1','bid_size2','ask_size2','size']].astype('int32')


In [58]:
s2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 57448800 entries, (5, 2, 0) to (32767, 126, 599)
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   bid_price1   float32
 1   ask_price1   float32
 2   bid_price2   float32
 3   ask_price2   float32
 4   bid_size1    int32  
 5   ask_size1    int32  
 6   bid_size2    int32  
 7   ask_size2    int32  
 8   price        float32
 9   size         int32  
 10  order_count  int16  
dtypes: float32(5), int16(1), int32(5)
memory usage: 2.5 GB


Next we will compute the Order price as a weighted aveare of the ask price, askisize, bid price, bis size as define above already with the formula. 

since we have two ask and bid details for each stock and time winodws, we will first take the average of the bid/ask price and the total bid/ask quanity. we will use the same into the WAP formula given below



https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data?cellId=18&cellIds=18&kernelSessionId=67183666

In [108]:
import IPython
url = 'https://www.kaggle.com/embed/jiashenliu/introduction-to-financial-concepts-and-data?cellId=18&cellIds=18&kernelSessionId=67183666'

#url = 'https://www.optiver.com/wp-content/uploads/2021/05/DataBucketing.png'

iframe = '<iframe src=' + url + ' width=700 height=350></iframe>'

IPython.display.HTML(iframe)

In [59]:
s2['order_wap'] = (((s2['bid_price1'] * s2['bid_size1'] + s2['bid_price2'] * s2['bid_size2'])/(s2['bid_size1'] + s2['bid_size2']) * (s2['ask_size1'] + s2['ask_size2'])) + ((s2['ask_price1'] * s2['ask_size1'] + s2['ask_price2'] * s2['ask_size2'])/(s2['ask_size1'] + s2['ask_size2'])) * (s2['bid_size1'] + s2['bid_size2'])) / ((s2['bid_size1'] + s2['bid_size2']) +  (s2['ask_size1'] + s2['ask_size2']))


In [60]:
s2['trade_wap'] = s2.groupby(['time_id','stock_id'])['price'].apply(lambda x: x - x.mean())
#s2['trade_wap_sq'] = s2.groupby(['time_id','stock_id'])['price'].apply(lambda x: x - x.mean())

In [61]:
pd.options.display.float_format = '{:,.16f}'.format

In [62]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

def trade_volatility(trade_wap_prices):
    return np.sqrt(np.sum(trade_wap_prices**2)/len(trade_wap_prices))

def order_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

In [63]:
s2['log_return'] = s2.groupby(['time_id','stock_id'])['order_wap'].apply(log_return)

In [64]:
s2['log_return'].isna().sum()

95748

In [65]:
#remove the rows with null values of log return
#s2[~s2['log_return'].isnull()]['log_return']
df_realized_vol_per_stock =  pd.DataFrame(s2[~s2['log_return'].isnull()]['log_return'].groupby(['time_id','stock_id']).agg(order_volatility))

In [66]:
df_realized_vol_per_stock.isna().sum()

log_return    0
dtype: int64

In [67]:
df_realized_vol_per_stock

log_return
time_id stock_id                   
5       2        0.0018731349846348
        5        0.0046019894070923
        9        0.0046338266693056
        13       0.0026797724422067
        14       0.0023218726273626
...                             ...
32767   109      0.0014521904522553
        110      0.0029210662469268
        111      0.0012556677684188
        114      0.0018005028832704
        126      0.0019010640680790

[95748 rows x 1 columns]

In [69]:
#df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock.apply(lambda x: fff(x.index.get_level_values(1),x.index.get_level_values(0)), axis=1 )
#df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock.apply(lambda x: x.index.get_level_values(1)+x.index.get_level_values(0))
df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock.apply(lambda x: x.index.get_level_values(1).astype(str) + "-" + x.index.get_level_values(0).astype(str))

#f'{stock_id}-{x}'

In [70]:
df_realized_vol_per_stock['trade_return'] = pd.DataFrame(s2['trade_wap'].groupby(['time_id','stock_id']).agg(trade_volatility))


In [71]:
df_realized_vol_per_stock

log_return     row_id       trade_return
time_id stock_id                                                 
5       2        0.0018731349846348        2-5 0.0005385612692736
        5        0.0046019894070923        5-5 0.0011968942817501
        9        0.0046338266693056        9-5 0.0019693513212417
        13       0.0026797724422067       13-5 0.0004783151735477
        14       0.0023218726273626       14-5 0.0003629519568591
...                             ...        ...                ...
32767   109      0.0014521904522553  109-32767 0.0003046170126018
        110      0.0029210662469268  110-32767 0.0001787947496083
        111      0.0012556677684188  111-32767 0.0003218932480909
        114      0.0018005028832704  114-32767 0.0005063665417761
        126      0.0019010640680790  126-32767 0.0003025906464201

[95748 rows x 3 columns]

In [33]:
# def past_realized_volatility_per_stock(list_file,prediction_column_name):
#     df_past_realized = pd.DataFrame()
#     for file in list_file:
#         df_past_realized = pd.concat([df_past_realized,
#                                      realized_volatility_per_time_id(file,prediction_column_name)])
#     return df_past_realized
# df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
#                                                            prediction_column_name='pred')

Let's join the output dataframe with train.csv to see the performance of the naive prediction on training set.

In [72]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]


We will evaluate the naive prediction result by two metrics: RMSPE and R squared.

In [73]:
df_joined = train.merge(df_realized_vol_per_stock, on = ['row_id'], how = 'inner')

In [74]:
df_joined

,row_id,target,log_return,trade_return
0,2-5,0.0018475460000000,0.0018731349846348,0.0005385612692736
1,2-11,0.0008059180000000,0.0008150119101629,0.0003309211217708
2,2-16,0.0015811100000000,0.0012938779545948,0.0002343745260598
3,2-31,0.0015986860000000,0.0015226512914523,0.0005046062067635
4,2-62,0.0015032210000000,0.0012423946755007,0.0002491856398152
...,...,...,...,...
95743,126-32751,0.0034606180000000,0.0036778114736080,0.0004475510012079
95744,126-32753,0.0031126940000000,0.0035839092452079,0.0011539259769618
95745,126-32758,0.0040696760000000,0.0033929920755327,0.0004800538980768
95746,126-32763,0.0033567180000000,0.0031998809427023,0.0003984458293942


In [75]:
from sklearn.metrics import r2_score, mean_squared_error
def rmspe(y_true, y_pred):
#    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100
    return mean_squared_error(y_true,y_pred,squared=False)

In [81]:
# R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['log_return']),3)
# RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['log_return']),3)
# print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}%')

In [82]:
# R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['log_return'] + np.sqrt(df_joined['trade_return'])),3)
# RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['log_return'] + np.sqrt(df_joined['trade_return'])),3)
# print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}%')

In [76]:
jovian.commit(filename="my-zero-to-gbm-proj-assign.ipynb")

NameError: name 'jovian' is not defined

## Machine Learning

Now we will start applying ML techniques to predict the volataility of the next 10 minutes window for each time-id/stock-id based on the order book volatility and trade volatility

we will learn the hyper parameters givne using the training targets for the same

we will use 2 different models to do the same

## Graident Bossting

We're now ready to train our gradient boosting machine (GBM) model. Here's how a GBM model works:

In [49]:
from xgboost import XGBRegressor

## Intel Extension for Scikit-learn

Intel(R) Extension for Scikit-learn* dynamically patches scikit-learn estimators to use Intel(R) oneAPI Data Analytics Library as the underlying solver, while getting the same solution faster.

To install these Intel-optimized packages for scikit-learn on Windows, Mac, and Linux x86_64, simply:

conda install scikit-learn-intelex

Once installed, there are two ways in which you can enable the replacement patching functionality for scikit-learn. You can enable it when you run your application:

python -m sklearnex my_application.py

Or you can explicitly enable the patching in your code:

from sklearnex import patch_sklearn

patch_sklearn()

In [50]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


### Cross Validation

 create a validation set before training our XGBoost model. We'll use a different validation strategy this time, called <b> ShuffleSplit </b> cross validation (source):

In [51]:
from sklearn.model_selection import ShuffleSplit

In [52]:
ss = ShuffleSplit(n_splits = 5, test_size = 0.25, random_state=42)

Let's define a helper function train_and_evaluate which trains a model the given parameters and returns the trained model, training error and validation error.

In [56]:
def train_and_evaluate(X_train, train_targets, X_val, val_targets, **params):
    model =XGBRegressor(random_state=11111,n_jobs=-1,validate_parameters=True,objective='reg:squarederror',max_depth=None,subsample=0.8,tree_method='approx',verbosity=1, **params)
    model.fit(X_train, train_targets)
    train_R2 = round(r2_score(train_targets, model.predict(X_train)),3)
    train_RMSPE = round(rmspe(train_targets, model.predict(X_train)),3)
    val_R2 = round(r2_score(val_targets, model.predict(X_val)),3)
    val_RMSPE = round(rmspe(val_targets, model.predict(X_val),),3)

    # train_rmse = rmse(model.predict(X_train), train_targets)
    # val_rmse = rmse(model.predict(X_val), val_targets)
    return model, train_R2, train_RMSPE, val_R2, val_RMSPE
#n_jobs = -1 means that use all the available threads in that machine where the alogorithm is running 

Now we will train the model for each split data of the ShuffleSplit

In [40]:
df_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95748 entries, 0 to 95747
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   row_id        95748 non-null  object 
 1   target        95748 non-null  float64
 2   log_return    95748 non-null  float32
 3   trade_return  95748 non-null  float64
dtypes: float32(1), float64(2), object(1)
memory usage: 3.3+ MB


In [41]:
inputs = df_joined[['log_return', 'trade_return']].copy().astype('float32')
targets = df_joined['target'].copy().astype('float32')

In [42]:
inputs.isna().sum(), targets.isna().sum()

(log_return      0
 trade_return    0
 dtype: int64,
 0)

In [43]:
inputs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95748 entries, 0 to 95747
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   log_return    95748 non-null  float32
 1   trade_return  95748 non-null  float32
dtypes: float32(2)
memory usage: 1.5 MB


In [44]:
targets

0       0.0018475459655747
1       0.0008059180108830
2       0.0015811099437997
3       0.0015986859798431
4       0.0015032209921628
               ...        
95743   0.0034606179688126
95744   0.0031126940157264
95745   0.0040696761570871
95746   0.0033567179925740
95747   0.0020903639961034
Name: target, Length: 95748, dtype: float32

In [63]:
# models = []

# for train_idxs, val_idxs in ss.split(inputs):
#     X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
#     X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
#     model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
#                                                      train_targets, 
#                                                      X_val, 
#                                                      val_targets, 
#                                                      max_depth=5, 
#                                                      n_estimators=50)
#     models.append(model)
#     print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

Let's also define a function to average predictions from the 5 different models.

In [45]:
def predict_avg(models, inputs):
    return np.mean([model.predict(inputs) for model in models], axis=0)

In [58]:
# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.experimental import enable_halving_search_cv
# now you can import normally from model_selection
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, HalvingGridSearchCV

In [59]:
model =XGBRegressor(random_state=11111,n_jobs=-1,validate_parameters=True,objective='reg:squarederror',max_depth=None,subsample=0.8,tree_method='approx',verbosity=1)
param_grid = {#"max_depth": [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, None],
              "n_estimators" : list(range(10,51,3)),
              "learning_rate": np.logspace(-3,3,50),
              "booster": ['gbtree', 'gblinear'],
              # "gamma": np.logspace(-1,1,100),
              # "subsample": np.linspace(0.5,1.0,9, endpoint=False),
              # "num_parallel_tree": range(1,6,1),
              # "reg_alpha": np.logspace(-3,3,100),
              # "reg_lambda": np.logspace(-3,3,100)
            }


In [61]:
models = []
for train_idxs, val_idxs in ss.split(inputs):
    X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
    X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
    X_train = np.ascontiguousarray(X_train).reshape(-1,2)
    train_targets = np.ascontiguousarray(train_targets).reshape(-1,1)
    X_val = np.ascontiguousarray(X_val).reshape(-1,2)
    val_targets = np.ascontiguousarray(val_targets).reshape(-1,1)
    grid_search = GridSearchCV(model, param_grid, n_jobs=-1,verbose=1).fit(X_train, np.ravel(train_targets))
    model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                    np.ravel(train_targets), 
                                                    X_val, 
                                                    np.ravel(val_targets), 
                                                    **grid_search.best_params_)
    models.append(model)
    print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))


Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [-29966.41975708 -29787.07431983 -29608.8023478  ...             nan
             nan             nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:929: RuntimeWarning: invalid value encountered in subtract
  array_stds = np.sqrt(np.average((array -


Train R2: 0.765, Train RMSPE: 0.0010000000474974513, Validation R2: 0.753, Validation RMSPE: 0.0010000000474974513
Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [-29793.15708523 -29614.84646366 -29437.60599766 ...             nan
             nan             nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:929: RuntimeWarning: invalid value encountered in subtract
  array_stds = np.sqrt(np.average((array -


Train R2: 0.766, Train RMSPE: 0.0010000000474974513, Validation R2: 0.751, Validation RMSPE: 0.0010000000474974513
Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


...

Train R2: 0.764, Train RMSPE: 0.0010000000474974513, Validation R2: 0.758, Validation RMSPE: 0.0010000000474974513
Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [-29799.48371971 -29621.13651364 -29443.85748119 ...             nan
             nan             nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:929: RuntimeWarning: invalid value encountered in subtract
  array_stds = np.sqrt(np.average((array -


Train R2: 0.763, Train RMSPE: 0.0010000000474974513, Validation R2: 0.759, Validation RMSPE: 0.0010000000474974513
Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [-29608.23390888 -29431.03288569 -29254.89036622 ...             nan
             nan             nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:929: RuntimeWarning: invalid value encountered in subtract
  array_stds = np.sqrt(np.average((array -


Train R2: 0.761, Train RMSPE: 0.0010000000474974513, Validation R2: 0.764, Validation RMSPE: 0.0010000000474974513


In [ ]:
# models = []
# for train_idxs, val_idxs in ss.split(inputs):
#     X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
#     X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
#     X_train = np.ascontiguousarray(X_train).reshape(-1,2)
#     train_targets = np.ascontiguousarray(train_targets).reshape(-1,1)
#     X_val = np.ascontiguousarray(X_val).reshape(-1,2)
#     val_targets = np.ascontiguousarray(val_targets).reshape(-1,1)
#     reg = LazyRegressor(ignore_warnings=False, random_state=11111, verbose=True)
#     models, predictions = reg.fit(X_train, X_val, train_targets, val_targets)  # pass all sets
#     print(models.head(100))


In [47]:
targets >= 0

0        True
1        True
2        True
3        True
4        True
         ... 
95743    True
95744    True
95745    True
95746    True
95747    True
Name: target, Length: 95748, dtype: bool

In [70]:
model = HistGradientBoostingRegressor(random_state=11111,scoring='neg_root_mean_squared_error',validation_fraction=0.2,verbose=1,loss='poisson')
param_grid = {"max_depth": [10, 20, 30, 40, 50, None],
              "max_leaf_nodes" : [15,31,63,127],
              "learning_rate": np.logspace(-3,3,50),
              "min_samples_leaf": [10,20,30],
              "l2_regularization": [0.0,0.00001,0.0001,0.001,0.01]
            }


In [48]:
def train_and_evaluate(X_train, train_targets, X_val, val_targets, **params):
    model = HistGradientBoostingRegressor(random_state=11111,scoring='neg_root_mean_squared_error',validation_fraction=0.2,verbose=1,loss='poisson', **params)
    model.fit(X_train, train_targets)
    train_R2 = round(r2_score(train_targets, model.predict(X_train)),3)
    train_RMSPE = round(rmspe(train_targets, model.predict(X_train)),3)
    val_R2 = round(r2_score(val_targets, model.predict(X_val)),3)
    val_RMSPE = round(rmspe(val_targets, model.predict(X_val),),3)

    # train_rmse = rmse(model.predict(X_train), train_targets)
    # val_rmse = rmse(model.predict(X_val), val_targets)
    return model, train_R2, train_RMSPE, val_R2, val_RMSPE
#n_jobs = -1 means that use all the available threads in that machine where the alogorithm is running 

In [88]:
# import sklearn.metrics
# sorted(sklearn.metrics.SCORERS.keys())

In [86]:
models = []
for train_idxs, val_idxs in ss.split(inputs):
    X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
    X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
    X_train = np.ascontiguousarray(X_train).reshape(-1,2)
    train_targets = np.ascontiguousarray(train_targets).reshape(-1,1)
    X_val = np.ascontiguousarray(X_val).reshape(-1,2)
    val_targets = np.ascontiguousarray(val_targets).reshape(-1,1)
    grid_search = RandomizedSearchCV(model, param_grid, random_state=1111, scoring='r2',verbose=1,return_train_score=True,n_jobs=-1,n_iter=20).fit(X_train,train_targets)
    model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                        train_targets, 
                                                        X_val, 
                                                        val_targets, 
                                                        **grid_search.best_params_)
    models.append(model)
    print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.6min finished


Binning 0.001 GB of training data: 0.109 s
Binning 0.000 GB of validation data: 0.022 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 15 leaves, max depth = 4, train score: -0.00214, val score: -0.00213, in 0.163s
[2/100] 1 tree, 15 leaves, max depth = 5, train score: -0.00180, val score: -0.00179, in 0.109s
[3/100] 1 tree, 15 leaves, max depth = 5, train score: -0.00161, val score: -0.00160, in 0.149s
[4/100] 1 tree, 15 leaves, max depth = 5, train score: -0.00150, val score: -0.00149, in 0.133s
[5/100] 1 tree, 15 leaves, max depth = 4, train score: -0.00144, val score: -0.00143, in 0.134s
[6/100] 1 tree, 15 leaves, max depth = 5, train score: -0.00140, val score: -0.00139, in 0.144s
[7/100] 1 tree, 15 leaves, max depth = 7, train score: -0.00139, val score: -0.00138, in 0.153s
[8/100] 1 tree, 15 leaves, max depth = 5, train score: -0.00138, val score: -0.00137, in 0.150s
[9/100] 1 tree, 15 leaves, max depth = 7, train score: -0.00137, val score: -0.00136, in 0.156s
[10/100] 1 tree

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   58.0s


Binning 0.001 GB of training data: 0.115 s
Binning 0.000 GB of validation data: 

...

0.016 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 15 leaves, max depth = 4, train score: -0.00214, val score: -0.00216, in 0.121s
[2/100] 1 tree, 15 leaves, max depth = 4, train score: -0.00180, val score: -0.00182, in 0.107s
[3/100] 1 tree, 15 leaves, max depth = 4, train score: -0.00160, val score: -0.00162, in 0.116s
[4/100] 1 tree, 15 leaves, max depth = 5, train score: -0.00150, val score: -0.00152, in 0.116s
[5/100] 1 tree, 15 leaves, max depth = 5, train score: -0.00144, val score: -0.00146, in 0.136s
[6/100] 1 tree, 15 leaves, max depth = 6, train score: -0.00141, val score: -0.00143, in 0.151s
[7/100] 1 tree, 15 leaves, max depth = 5, train score: -0.00139, val score: -0.00141, in 0.156s
[8/100] 1 tree, 15 leaves, max depth = 5, train score: -0.00138, val score: -0.00140, in 0.169s
[9/100] 1 tree, 15 leaves, max depth = 6, train score: -0.00138, val score: -0.00140, in 0.152s
[10/100] 1 tree, 15 leaves, max depth = 6, train score: -0.00138, val score: -0.00140, in 0.139

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   46.7s


Binning 0.001 GB of training data: 0.111 s
Binning 0.000 GB of validation data: 

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.1min finished


0.019 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 15 leaves, max depth = 4, train score: -0.00219, val score: -0.00217, in 0.103s
[2/100] 1 tree, 15 leaves, max depth = 4, train score: -0.00184, val score: -0.00182, in 0.118s
[3/100] 1 tree, 15 leaves, max depth = 4, train score: -0.00164, val score: -0.00163, in 0.143s
[4/100] 1 tree, 15 leaves, max depth = 5, train score: -0.00153, val score: -0.00152, in 0.133s
[5/100] 1 tree, 15 leaves, max depth = 5, train score: -0.00147, val score: -0.00146, in 0.125s
[6/100] 1 tree, 15 leaves, max depth = 6, train score: -0.00143, val score: -0.00143, in 0.182s
[7/100] 1 tree, 15 leaves, max depth = 6, train score: -0.00142, val score: -0.00141, in 0.159s
[8/100] 1 tree, 15 leaves, max depth = 6, train score: -0.00141, val score: -0.00140, in 0.173s
[9/100] 1 tree, 15 leaves, max depth = 8, train score: -0.00140, val score: -0.00139, in 0.165s
[10/100] 1 tree, 15 leaves, max depth = 6, train score: -0.00140, val score: -0.00139, in 0.169

In [87]:
grid_search.best_params_

{'min_samples_leaf': 30,
 'max_leaf_nodes': 15,
 'max_depth': None,
 'learning_rate': 0.28117686979742307,
 'l2_regularization': 0.0}

In [ ]:
preds = predict_avg(models, inputs)

In [49]:
model = HistGradientBoostingRegressor(random_state=11111,scoring='neg_root_mean_squared_error',validation_fraction=0.2,verbose=1,loss='least_squares')
param_grid = {#"max_depth": [10, 20, 30, 40, 50, None],
              "max_leaf_nodes" : [7,15,31,63],
              "learning_rate": np.logspace(-1,1,10),
              "min_samples_leaf": [20,30,40,50],
              #"l2_regularization": [0.0,0.00001,0.0001,0.001,0.01]
            }


In [50]:
models = []
for train_idxs, val_idxs in ss.split(inputs):
    X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
    X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
    X_train = np.ascontiguousarray(X_train).reshape(-1,2)
    train_targets = np.ascontiguousarray(train_targets).reshape(-1,1)
    X_val = np.ascontiguousarray(X_val).reshape(-1,2)
    val_targets = np.ascontiguousarray(val_targets).reshape(-1,1)
    grid_search = GridSearchCV(model, param_grid,  scoring='r2',verbose=1,return_train_score=True,n_jobs=-1).fit(np.X_train,train_targets)
    model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
                                                        train_targets, 
                                                        X_val, 
                                                        val_targets, 
                                                        **grid_search.best_params_)
    models.append(model)
    print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))


Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  3.7min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Binning 0.001 GB of training data: 0.134 s
Binning 0.000 GB of validation data: 0.024 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00248, val score: -0.00248, in 0.150s
[2/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00221, val score: -0.00221, in 0.126s
[3/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00200, val score: -0.00200, in 0.166s
[4/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00184, val score: -0.00183, in 0.146s
[5/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00172, val score: -0.00171, in 0.182s
[6/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00162, val score: -0.00162, in 0.340s
[7/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00156, val score: -0.00155, in 0.204s
[8/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00150, val score: -0.00149, in 0.182s
[9/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00147, val score: -0.00146, in 0.154s
[10/100] 1 tree, 7 leave

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.117 s
Binning 0.000 GB of validation data: 0.021 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00245, val score: -0.00244, in 0.092s
[2/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00217, val score: -0.00217, in 0.136s
[3/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00196, val score: -0.00196, in 0.140s
[4/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00181, val score: -0.00180, in 0.153s
[5/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00169, val score: -0.00168, in 0.159s
[6/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00160, val score: -0.00160, in 0.161s
[7/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00154, val score: -0.00153, in 0.149s
[8/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00149, val score: -0.00149, in 0.191s
[9/100] 1 tree, 7 leaves, max depth = 4, train score: -0.00146, val score: -0.00145, in 0.178s
[10/100] 1 tree, 7 leaves, max depth = 3, train score: -0.0

...

Binning 0.001 GB of training data: 0.098 s
Binning 0.000 GB of validation data: 

[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  5.4min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: RuntimeWarning: invalid value encountered in subtract
  array_stds = np.sqrt(np.average((array -
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.028 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00267, val score: -0.00264, in 0.194s
[2/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00248, val score: -0.00245, in 0.192s
[3/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00232, val score: -0.00229, in 0.180s
[4/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00218, val score: -0.00215, in 0.196s
[5/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00206, val score: -0.00203, in 0.189s
[6/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00196, val score: -0.00193, in 0.207s
[7/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00187, val score: -0.00184, in 0.179s
[8/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00180, val score: -0.00177, in 0.206s
[9/100] 1 tree, 7 leaves, max depth = 4, train score: -0.00173, val score: -0.00170, in 0.210s
[10/100] 1 tree, 7 leaves, max depth = 4, train score: -0.00168, val score: -0.00165, in 0.192s
[11/100]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.023 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00267, val score: -0.00264, in 0.056s
[2/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00248, val score: -0.00245, in 0.053s
[3/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00232, val score: -0.00229, in 0.081s
[4/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00218, val score: -0.00215, in 0.090s
[5/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00206, val score: -0.00203, in 0.108s
[6/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00196, val score: -0.00193, in 0.096s
[7/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00187, val score: -0.00184, in 0.087s
[8/100] 1 tree, 7 leaves, max depth = 3, train score: -0.00180, val score: -0.00177, in 0.102s
[9/100] 1 tree, 7 leaves, max depth = 4, train score: -0.00173, val score: -0.00170, in 0.082s
[10/100] 1 tree, 7 leaves, max depth = 4, train score: -0.00168, val score: -0.00165, in 0.108s
[11/100]

Discussion on cross validation designs

https://www.kaggle.com/vishnurapps/undersanding-kfold-stratifiedkfold-and-groupkfold